In [1]:
using Pkg
pkg"activate ."

In [2]:
using PerlaTonettiWaugh, LinearAlgebra, Plots, BenchmarkTools, Interpolations, QuadGK, DifferentialEquations, BlackBoxOptim, Optim, DataFrames

# Steady states

## Solving steady state solutions

Set up parameters and find the corresponding stationary solution:

In [ ]:
# Define common objects. 
params = parameter_defaults()
settings = settings_defaults()
d_0 = 3.0700
d_T = 2.5019
params_0 = merge(params, (d = d_0,)) # parameters to be used at t = 0
params_T = merge(params, (d = d_T,)) # parameters to be used at t = T
z_grid = settings.z
M = length(z_grid)

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_numerical(params_0, z_grid) # solution at t = 0
stationary_sol = stationary_numerical(params_T, z_grid) # solution at t = T

Ω_0 = stationary_sol_0.Ω
Ω_T = stationary_sol.Ω

## Welfare in steady states

In [ ]:
# auxil. function that extracts and displays welfare information from steady state solutions
function display_stationary_sol(stationary_sol)
    @unpack g, z_hat, Ω, π_bar_agg, y, c, λ_ii, U_bar, L_tilde, z_bar, w, x, π_min, r, a, b, S = stationary_sol
    @show g
    @show z_hat
    @show Ω
    @show π_bar_agg
    @show y
    @show c
    @show λ_ii
    @show U_bar(0.0)
    @show L_tilde
    @show z_bar
    @show w
    @show x
    @show π_min
    @show r
    @show a
    @show b
    @show S
end;

### Steady state at `T`

In [ ]:
display_stationary_sol(stationary_sol);

### Steady state at `0`

In [ ]:
display_stationary_sol(stationary_sol_0);

# Transition dynamics

Setup for optimizer:

In [ ]:
ENTRY_RESIDUALS_NODES_COUNT = 15
E_NODE_COUNT = 15
SOLUTION = [-0.9283233397558605, -0.7931227777335486, -0.6049356364806617, -0.4927044449099116, -0.3358353090972862, -0.2497406170581776, -0.24267972513426644, -0.1251863870906156, -0.0948087489326721, -0.08776386898955374, -0.05730272455803215, -0.04512884607138278, -0.030714141187059757, -0.03010019595942575, 39.420517045082555]
RANGES_NODES = map(i->(SOLUTION[i] - 0.0, min(-1e-3, SOLUTION[i] + 0.0)), 1:(E_NODE_COUNT-1))
RANGES_T = (SOLUTION[end], SOLUTION[end])
RANGES = [RANGES_NODES; RANGES_T]
WEIGHTS = [fill(15, 3); fill(1, ENTRY_RESIDUALS_NODES_COUNT-3)]
ITERATIONS = 2
SORT_CANDIDATE = true # this will make finding solution much more efficient when we use global optimization schemes

settings_optim = (params_T = params_T, stationary_sol_T = stationary_sol, Ω_0 = Ω_0,
                E_node_count = E_NODE_COUNT, entry_residuals_nodes_count = ENTRY_RESIDUALS_NODES_COUNT, 
                weights = WEIGHTS, ranges = RANGES, iterations = ITERATIONS,
                sort_candidate = SORT_CANDIDATE)
settings = merge(settings, settings_optim);

Find the corresponding solution:

In [ ]:
result = solve_full_model_global(SOLUTION, settings);
solved = result.solution;
E_nodes_and_T = result.E_nodes_and_T;

In [ ]:
println("E_nodes (sorted) : ")
sort(E_nodes_and_T[1:(length(E_nodes_and_T)-1)])

In [ ]:
println("T: ")
E_nodes_and_T[end]

## Plots for Ω and `entry_residuals`

In [ ]:
v_t0 = solved.sol.u[1][1:M]
v0 = solved.results[:v_0]
# save v0 and v_hat_t0
v_hat_t0 = map(z -> exp((params.σ-1)*z), z_grid) .* v_t0;

solved = solved.results;
plot_Ω = plot(solved.t, solved.Ω, label = "Omega", lw = 3)
plot_residual = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot_Ω, plot_residual, layout = (2,1))

## Primary Plots

In [ ]:
plot1 = plot(solved.t, solved.g, label = "g", lw = 3)
plot2 = plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)
plot3 = plot(solved.t, solved.S, label = "S", lw = 3)
plot4 = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot1, plot2, plot3, plot4, layout=(2,2))

## Static Equations

In [ ]:
plot1 = plot(solved.t, solved.L_tilde, label = "L_tilde", lw = 3)
plot2 = plot(solved.t, solved.z_bar, label = "z_bar", lw = 3)
plot3 = plot(solved.t, solved.π_min, label = "pi_min", lw = 3)
plot4 = plot(solved.t, solved.λ_ii, label = "lambda_ii", lw = 3)
plot5 = plot(solved.t, solved.c, label = "c", lw = 3)
plot6 = plot(solved.t, solved.E, label = "E", lw = 3)
plot7 = plot(solved.t, solved.log_M, label = "log_M", lw = 3)
plot8 = plot(solved.t, solved.U, label = "U", lw = 3)
plot9 = plot(z_grid, v_hat_t0, label = "v_hat at t = 0", lw = 3)
plot(plot1, plot2, plot3, plot4, plot5, plot6, plot7, plot8, layout=(4,2))

# Welfare analysis

### `log_M(t)`

In [ ]:
# define log_M with steady state g
log_M_steady_0(t) = stationary_sol_0.g * t
log_M_steady_T(t) = stationary_sol.g * t

# generate the plot!
plot(solved.t, 
    [solved.log_M, log_M_steady_0, log_M_steady_T], 
    label = ["log_M_transition_0", "log_M_steady_0", "log_M_steady_T"] ,
    title = "log_M(t)", linestyle = :auto, lw = 3)

### `Ω(t)`

In [ ]:
# define Ω function (constant)
Ω_steady_0(t) = stationary_sol_0.Ω
Ω_steady_T(t) = stationary_sol.Ω

# generate the plot!
plot(solved.t, 
    [solved.Ω, Ω_steady_0, Ω_steady_T], 
    label = ["Omega_transition_0", "Omega_steady_0", "Omega_steady_T"] ,
    title = "Omega(t)", linestyle = :auto, lw = 3)

### `c(t)`

In [ ]:
# define c function (constant)
c_steady_0(t) = stationary_sol_0.c
c_steady_T(t) = stationary_sol.c

# generate the plot!
plot(solved.t, 
    [solved.c, c_steady_0, c_steady_T], 
    label = ["c_transition_0", "c_steady_0", "c_steady_T"] ,
    title = "c(t)", linestyle = :auto, lw = 3)

### `U(t)`

In [ ]:
# generate the plot!
plot(solved.t, 
    [solved.U, stationary_sol_0.U_bar, stationary_sol.U_bar], 
    label = ["U_transition_0", "U_steady_0", "U_steady_T"] ,
    title = "U(t)", linestyle = :auto, lw = 3)

## Consumption equivalent for search threshold (`M(0)`)
See computational appendix for details.

### `M(0)` by two steady states (autarky and steady state at `T`)

In [ ]:
T = solved.t[end]
M_0 = exp(params.ρ*(stationary_sol.U_bar(T) - stationary_sol_0.U_bar(T)))

### `M(0)` by autarky and transition from `t=0`

In [ ]:
T = solved.t[end]
M_0 = exp(params.ρ*(solved.U[1] - stationary_sol_0.U_bar(T)))